# Signal Processing Basics

This notebook demonstrates classical signal processing techniques available in Promethium.

**Prerequisites:**
- Python 3.10+
- Basic understanding of signal processing

**Estimated Runtime:** 5 minutes

**Topics Covered:**
- Bandpass, lowpass, and highpass filtering
- Notch filtering for noise removal
- Time-frequency analysis

In [ ]:
# Uncomment to install from PyPI:
# !pip install promethium-seismic==1.0.1

In [ ]:
import promethium
from promethium import (
    bandpass_filter,
    lowpass_filter,
    highpass_filter,
    notch_filter,
    generate_synthetic_traces,
    add_noise,
    set_seed,
)

import numpy as np
import matplotlib.pyplot as plt
from scipy import signal as scipy_signal

print(f"Promethium version: {promethium.__version__}")
set_seed(42)

## 1. Generate Test Data

In [ ]:
# Generate synthetic traces with specific frequency content
clean_data, metadata = generate_synthetic_traces(
    n_traces=10,
    n_samples=2000,
    sample_rate=500.0,
    frequencies=[5.0, 20.0, 50.0],
    seed=42
)

# Add noise
noisy_data = add_noise(clean_data, noise_level=0.2, seed=42)

# Select a single trace for demonstration
trace = noisy_data[0]
fs = metadata['sample_rate']
t = np.arange(len(trace)) / fs

print(f"Trace length: {len(trace)} samples")
print(f"Sample rate: {fs} Hz")
print(f"Duration: {len(trace)/fs:.2f} seconds")

## 2. Frequency Analysis

Compute and visualize the frequency spectrum of the signal.

In [ ]:
def plot_spectrum(signal, fs, title="Frequency Spectrum", ax=None):
    """Plot the frequency spectrum of a signal."""
    n = len(signal)
    freqs = np.fft.rfftfreq(n, 1/fs)
    spectrum = np.abs(np.fft.rfft(signal))
    spectrum_db = 20 * np.log10(spectrum + 1e-10)
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 4))
    
    ax.plot(freqs, spectrum_db, 'b-', linewidth=0.8)
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('Magnitude (dB)')
    ax.set_title(title)
    ax.grid(True, alpha=0.3)
    ax.set_xlim([0, fs/2])
    
    return ax

# Plot time domain and frequency domain
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

axes[0].plot(t, trace, 'b-', linewidth=0.5)
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Amplitude')
axes[0].set_title('Time Domain Signal')
axes[0].grid(True, alpha=0.3)

plot_spectrum(trace, fs, 'Frequency Spectrum', axes[1])

plt.tight_layout()
plt.show()

## 3. Bandpass Filtering

Apply a bandpass filter to isolate a specific frequency range.

In [ ]:
# Apply bandpass filter (10-40 Hz passband)
lowcut = 10.0
highcut = 40.0

filtered_bandpass = bandpass_filter(
    trace,
    lowcut=lowcut,
    highcut=highcut,
    fs=fs,
    order=5,
    zero_phase=True
)

# Compare original and filtered
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Time domain - Original
axes[0, 0].plot(t, trace, 'b-', linewidth=0.5)
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('Amplitude')
axes[0, 0].set_title('Original Signal')
axes[0, 0].grid(True, alpha=0.3)

# Time domain - Filtered
axes[0, 1].plot(t, filtered_bandpass, 'g-', linewidth=0.5)
axes[0, 1].set_xlabel('Time (s)')
axes[0, 1].set_ylabel('Amplitude')
axes[0, 1].set_title(f'Bandpass Filtered ({lowcut}-{highcut} Hz)')
axes[0, 1].grid(True, alpha=0.3)

# Spectrum - Original
plot_spectrum(trace, fs, 'Original Spectrum', axes[1, 0])
axes[1, 0].axvline(lowcut, color='r', linestyle='--', label='Passband')
axes[1, 0].axvline(highcut, color='r', linestyle='--')
axes[1, 0].legend()

# Spectrum - Filtered
plot_spectrum(filtered_bandpass, fs, 'Filtered Spectrum', axes[1, 1])
axes[1, 1].axvline(lowcut, color='r', linestyle='--', label='Passband')
axes[1, 1].axvline(highcut, color='r', linestyle='--')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 4. Lowpass and Highpass Filtering

In [ ]:
# Apply lowpass filter (remove high frequencies)
cutoff_low = 30.0
filtered_lowpass = lowpass_filter(trace, cutoff=cutoff_low, fs=fs)

# Apply highpass filter (remove low frequencies)
cutoff_high = 15.0
filtered_highpass = highpass_filter(trace, cutoff=cutoff_high, fs=fs)

fig, axes = plt.subplots(3, 2, figsize=(14, 10))

# Lowpass
axes[0, 0].plot(t, filtered_lowpass, 'b-', linewidth=0.5)
axes[0, 0].set_title(f'Lowpass Filtered (< {cutoff_low} Hz)')
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].grid(True, alpha=0.3)

plot_spectrum(filtered_lowpass, fs, ax=axes[0, 1])
axes[0, 1].axvline(cutoff_low, color='r', linestyle='--', label='Cutoff')
axes[0, 1].legend()

# Highpass
axes[1, 0].plot(t, filtered_highpass, 'g-', linewidth=0.5)
axes[1, 0].set_title(f'Highpass Filtered (> {cutoff_high} Hz)')
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].grid(True, alpha=0.3)

plot_spectrum(filtered_highpass, fs, ax=axes[1, 1])
axes[1, 1].axvline(cutoff_high, color='r', linestyle='--', label='Cutoff')
axes[1, 1].legend()

# Original for comparison
axes[2, 0].plot(t, trace, 'k-', linewidth=0.5, alpha=0.7)
axes[2, 0].set_title('Original Signal')
axes[2, 0].set_xlabel('Time (s)')
axes[2, 0].grid(True, alpha=0.3)

plot_spectrum(trace, fs, ax=axes[2, 1])

plt.tight_layout()
plt.show()

## 5. Spectrogram Analysis

In [ ]:
# Compute and plot spectrogram
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Original spectrogram
f, t_spec, Sxx = scipy_signal.spectrogram(trace, fs, nperseg=256, noverlap=128)
axes[0].pcolormesh(t_spec, f, 10*np.log10(Sxx + 1e-10), shading='gouraud', cmap='viridis')
axes[0].set_ylabel('Frequency (Hz)')
axes[0].set_title('Original Signal Spectrogram')

# Filtered spectrogram
f, t_spec, Sxx = scipy_signal.spectrogram(filtered_bandpass, fs, nperseg=256, noverlap=128)
im = axes[1].pcolormesh(t_spec, f, 10*np.log10(Sxx + 1e-10), shading='gouraud', cmap='viridis')
axes[1].set_ylabel('Frequency (Hz)')
axes[1].set_xlabel('Time (s)')
axes[1].set_title('Bandpass Filtered Spectrogram')

plt.colorbar(im, ax=axes, label='Power (dB)')
plt.tight_layout()
plt.show()

## 6. Summary

This notebook demonstrated:

1. **Frequency Analysis**: Computing and visualizing signal spectra
2. **Bandpass Filtering**: Isolating specific frequency ranges
3. **Lowpass/Highpass Filtering**: Removing high or low frequency content
4. **Spectrogram Analysis**: Time-frequency representation

### Next Steps

- **04_matrix_completion_and_compressive_sensing.ipynb**: Classical recovery methods
- **05_deep_learning_unet_reconstruction.ipynb**: Deep learning inference